<a href="https://www.kaggle.com/code/emanoelcarvalholopes/uci-dt-studentsperformanceprediction?scriptVersionId=214462108" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.tree import export_graphviz
import graphviz

# Obtenção dos Dados

In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
students = fetch_ucirepo(id=320) 

# Junção das duas bases de dados 

# students.data.features + students_data.targets 

students_full = pd.merge(students.data.features,students.data.targets, left_index = True, right_index = True, how = 'inner')

In [3]:
# students_full.head()

### Investigação Preliminar

In [4]:
# Separar a classe alvo das variáveis independentes

y = students_full.G3

X = students_full.iloc[:,:-1] # atributos do início até o penúltimo

# Apenas numéricos nos dados de treinamento
X = X.select_dtypes(exclude=['object'])


In [5]:
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X,y,train_size=0.8, test_size=0.2, random_state=1)

In [6]:
modelo = DecisionTreeClassifier(random_state=1)
modelo.fit(X_treinamento, y_treinamento)

DecisionTreeClassifier(random_state=1)

In [7]:
previsoes = modelo.predict(X_teste)

In [8]:
accuracy = accuracy_score(y_teste, previsoes)
precision = precision_score(y_teste, previsoes, average='weighted', zero_division=0)
recall = recall_score(y_teste, previsoes, average='weighted')
f1 = f1_score(y_teste, previsoes, average='weighted')
print(f'Acurácia: {accuracy}, Precisão: {precision}, Recall: {recall}, F1: {f1}')

Acurácia: 0.4230769230769231, Precisão: 0.4444061820532409, Recall: 0.4230769230769231, F1: 0.42476553111124493


Observação: acurácia baixa, esse modelo precisa ser aprimorado.